# The ensemble approach is to take the data, and train each of the N classifiers on a different random arrangement of 6 landmarks. 

In [9]:
# first get all X and y data from the all_points_folds
import numpy as np 
import pickle, random 
import cv2

X, y = [], []

for file in os.listdir("all_points_folds"): 
    with open(f"all_points_folds/{file}", 'rb') as f: 
        X_y = pickle.load(f)
        X.append(X_y[0])
        y.append(X_y[1]) 

In [10]:
import random 
def shuffle(X, y, seed = None):
    if seed == None:  
        seed = random.randrange(0, 100)
        print(f"using seed {seed}")
    np.random.seed(seed) 
    new_X = np.concatenate([X_i for X_i in X])
    new_y = np.concatenate([y_i for y_i in y])
    N = np.random.permutation(new_X.shape[0])
    new_X = new_X[N]
    new_y = new_y[N]
    new_X = new_X.reshape(5, 20, 90, 126)
    new_y = new_y.reshape(5, 20)
    return new_X, new_y
X, y = shuffle(X, y, seed = 68)

In [17]:
# create the randomly generated hand arrangments for 200 models

NUM_ARRANGEMENTS = 10
hand_arrangements = [] # (NUM_ARRANGEMENTS, 6) matrix

for arrangement in range(NUM_ARRANGEMENTS): 
    hand_arrangements.append(random.sample(list(range(0, 21)), 6))

def generate_sub_X(hand_arrangement, X): 
    columns_of_landmarks = [] # to access 
    for arrangement in hand_arrangement: 
        for start_point in [0, 21, 42, 63, 84, 105]: 
            columns_of_landmarks.append(start_point + arrangement) 
    return X.reshape(100, 90, 126)[:, :, tuple(columns_of_landmarks)] 

In [18]:
Xs = []
for hand_arrangement in hand_arrangements:
    Xs.append(generate_sub_X(hand_arrangement, X))

In [36]:
import tensorflow as tf 
def make_model(): 
    model = tf.keras.models.Sequential([
        tf.keras.layers.LSTM(64, return_sequences=False), 
        tf.keras.layers.Dropout(0.3), 
        tf.keras.layers.Dense(1, activation = "sigmoid")
    ])

    model.compile(loss = "binary_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005), metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
    
    return model 

def train_models():
    models = [] 
    checkpoint = tf.keras.callbacks.ModelCheckpoint("best_ensemble.h5", save_best_only=True, monitor = "val_accuracy")
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor = "val_accuracy", patience=10)
    callbacks = [checkpoint, early_stopping] 
    for i in range(len(hand_arrangements)): 
        model = make_model() 
        temp_X = Xs[i] 

        temp_X = temp_X.reshape(5, 20, 90, 36)
        X_test, y_test = temp_X[4], y[4]
        X_train = np.concatenate([X_j for j, X_j in enumerate(temp_X) if 4 !=j])
        y_train = np.concatenate([y_j for j, y_j in enumerate(y) if 4 !=j ]) 

        print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
        # the next thing after this is to train the model 
        model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 75, callbacks = callbacks)
        model.load_weights("best_ensemble.h5")
        model.evaluate(X_test, y_test)

        models.append(model)
    return models

In [37]:
models = train_models() 

(80, 90, 36) (20, 90, 36) (80,) (20,)
Epoch 1/75
3/3 [==============================] - 2s 260ms/step - loss: 0.6990 - accuracy: 0.6625 - precision_5: 0.5926 - recall_5: 0.8649 - val_loss: 0.6667 - val_accuracy: 0.7000 - val_precision_5: 0.7692 - val_recall_5: 0.7692
Epoch 2/75
3/3 [==============================] - 0s 29ms/step - loss: 0.6787 - accuracy: 0.7125 - precision_5: 0.6522 - recall_5: 0.8108 - val_loss: 0.6697 - val_accuracy: 0.6500 - val_precision_5: 0.7500 - val_recall_5: 0.6923
Epoch 3/75
3/3 [==============================] - 0s 29ms/step - loss: 0.6708 - accuracy: 0.7750 - precision_5: 0.7436 - recall_5: 0.7838 - val_loss: 0.6697 - val_accuracy: 0.7000 - val_precision_5: 0.8889 - val_recall_5: 0.6154
Epoch 4/75
3/3 [==============================] - 0s 29ms/step - loss: 0.6643 - accuracy: 0.7625 - precision_5: 0.7500 - recall_5: 0.7297 - val_loss: 0.6691 - val_accuracy: 0.6500 - val_precision_5: 0.8750 - val_recall_5: 0.5385
Epoch 5/75
3/3 [=============================

In [45]:
def aggregate_accuracy(Xs, models): 
    # get X_test for each model 
    preds = np.zeros((20, 1)) 
    for X, model in zip(Xs, models): 
        temp_X = X.reshape(5, 20, 90, 36) 
        X_test, y_test = temp_X[4], y[4]
        preds += model.predict(X_test)
    return preds/len(models)

In [52]:
np.round_(aggregate_accuracy(Xs, models).flatten()) == y[4]

array([False,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True, False,  True,  True,  True, False, False,  True,
        True,  True])

0.8